## Descriptive analysis

In [ ]:
from __future__ import print_function


import numpy as np
import scipy.io as sio
from scipy import stats
import matplotlib.pyplot as plt



import numpy as np
from sklearn.cluster import KMeans

import numpy as np
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.neural_network import MLPRegressor


from __future__ import print_function

import os
import csv
import numpy as np
import scipy.io as sio
from scipy import stats
import pandas as pd
from sklearn import linear_model
from sklearn.metrics import r2_score
import KMean_MissingImpute as KMI
import ModelEva

In [ ]:


print('Loading data...')
datafit = sio.loadmat('datafit.mat')
X = datafit['X']
Y = np.squeeze(datafit['Y'])
#Y = datafit['Y']

print('Loaded X matrix shape:', X.shape)
print('Loaded Y vector shape:', Y.shape)

mCovX = np.corrcoef(X.T)

print('Y describe', stats.describe(Y, axis=0))
nGlobalMin = np.amin(Y) - 0.001
print('Y describe', stats.describe(np.log(Y - nGlobalMin), axis=0))
nQu = np.percentile(Y, 0.1)
nQl = np.percentile(Y,99.9)
index = np.where((Y>nQu) * (Y<nQl))
Y = Y[index]
X = X[index]
dR = stats.describe(Y, axis=0)
mDesarray = np.array([[dR.mean, dR.variance, dR.skewness, dR.kurtosis]])

for iCol in range(8):
    Xp = X[:, iCol]
    Xp = Xp[~np.isnan(Xp)]

    plt.boxplot(Xp)
    plt.show()
    print('Col num %d' %iCol, stats.describe(Xp, axis=0))
    dR = stats.describe(Xp, axis=0)
    #mDesarray = np.concatenate((mDesarray, np.array([[dR.mean, dR.variance, dR.skewness, dR.kurtosis]])), axis=0)
np.savetxt('Description.csv', mDesarray, delimiter=',')

## Missing value imputation

In [ ]:


def kmeans_missing(X, n_clusters, max_iter=5):
    """Perform K-Means clustering on data with missing values.

    Args:
      X: An [n_samples, n_features] array of data to cluster.
      n_clusters: Number of clusters to form.
      max_iter: Maximum number of EM iterations to perform.

    Returns:
      labels: An [n_samples] vector of integer labels.
      centroids: An [n_clusters, n_features] array of cluster centroids.
      X_hat: Copy of X with the missing values filled in.
    """

    # Initialize missing values to their column means
    missing = np.isnan(X)
    mu = np.nanmean(X, 0, keepdims = 1)
    mu[np.isnan(mu)] = 0 #If all of one column is 0, just set it to 0
    X_hat = np.where(missing, mu, X)

    for i in range(max_iter):
        if i > 0:
            # initialize KMeans with the previous set of centroids. this is much
            # faster and makes it easier to check convergence (since labels
            # won't be permuted on every iteration), but might be more prone to
            # getting stuck in local minima.
            cls = KMeans(n_clusters, init= prev_centroids )
        else:
            # do multiple random initializations in parallel
            cls = KMeans(n_clusters, n_jobs=1)

        # perform clustering on the filled-in data
        labels = cls.fit_predict(X_hat)
        centroids = cls.cluster_centers_

        # fill in the missing values based on their cluster centroids
        X_hat[missing] = centroids[labels][missing]

        # when the labels have stopped changing then we have converged
        if i > 0 and np.all(labels == prev_labels):
            break

        prev_labels = labels
        prev_centroids = cls.cluster_centers_

    return labels, centroids, X_hat

## Model evaluation

In [ ]:


def fLoss(vTure, vPredict, nAggre = 2, nConserv = 1):
    #vTure: true return vector
    #vPredict: Predicted return vector
    #nAggre: The penalty coefficient if the prediction is aggressive
    #nConserv: The penalty coefficient if the prediction is conservative

    #Returns the mean loss score of the prediction. This value is always non-nagetive

    vPercentErr = (vTure - vPredict) #/ (vTure) #For percentage bias
    iIndex = vPercentErr <= 0 #The true return is less than predicted return. So our prediction is too aggresive
    vPercentErr[iIndex] *= -nAggre
    iIndex = vPercentErr > 0
    vPercentErr[iIndex] *= nConserv

    return np.mean(vPercentErr)

def fModelSelection(sModel = 'Linear'):
    #sModel: Choose different model

    #Return model object

    if sModel == 'Linear':# Linear model for OLS
        model = linear_model.LinearRegression()
    elif sModel == 'SVR': #SVM regressing
        dParameters = {'kernel':['linear','rbf'],'C':[3,5,9], 'gamma':[0.03,0.05,0.1]} # 'poly' 'degree':[2,3,4],
        model = GridSearchCV(SVR(), dParameters)
    elif sModel == 'EN': # Elastic Net
        dParameters = {'alpha': [1, 1.5, 2], 'l1_ratio' : [0.3, 0.5, 0.7]}
        model = GridSearchCV(ElasticNet(random_state=0), dParameters)
    elif sModel == 'MLP': # Multilayer perceptron
        dParameters = {'alpha': [0.0001,0.001, 0.01, 0.1]}
        model = GridSearchCV(MLPRegressor(), dParameters)
    else:#Exception handeling
        raise ValueError('Invalide model argument.for sModel! Your sModel is: %s' %sModel)
    return model


def fForwardSelectFeature(mX,vY, sScore = 'adj_r2', nAggre = 2, nConserv = 1, sModel = 'Linear'):
    #X: numpy array (matrix), independent variables with one column for one feature
    #Y: numpy array (vector), dependent variables
    #sScore: The criteria for selection, the higher the better
    #    fLoss: Loss function I defined above
    #    adj_r2: Adjusted R square

    #Returns (current model, a list shows the feature# (column#) selected by the forwarding method, and the finial score)

    lSelectedFea = []
    lCandidateFea = list(range(mX.shape[1]))
    nCurrentScore = -1000000.0
    nBestScore = -1000000.0

    model = fModelSelection(sModel)
    """
    """
    #In  we do not want the loop to select features
    if sModel == ' ':
        model.fit(mX, vY)#This step is really slow
        #print(model.best_estimator_.coef_)#For grid search
        #print(model.coefs_)#For MLP

        vYhat = model.predict(mX)
        print('Parameters grid search')
        if sScore == 'fLoss':
            nScore = -fLoss(vY, vYhat, nAggre, nConserv)  # The highr the better. So negative
        elif sScore == 'adj_r2':
            nScore = r2_score(vY, vYhat)
            print('r2: ', nScore)
            # nScore = model.score(mXpart, vY)
            nScore = 1.0 - ((1.0 - nScore) * (mX.shape[0] - 1.0) / ( mX.shape[0] - len(lCandidateFea) - 1.0))  # Adj R2 = 1 - (1 - R2) * (n-1)/n-k-1
            print('adj_r2: ', nScore)
        else:  # Exception handeling
            raise ValueError('Invalide model argument.for sScore! Your sScore is: %s' % sScore)
        return model, lCandidateFea, nScore


    while lCandidateFea and nCurrentScore == nBestScore:
        lCandidateScores = []
        for nCandidateFea in lCandidateFea:
            lFeatures = lSelectedFea + [nCandidateFea]
            mXpart = np.copy(mX[:,lFeatures ])
            model.fit(mXpart, vY)
            vYhat = model.predict(mXpart)
            #print('vYhat shape: ', vYhat.shape)

            if sScore == 'fLoss':
                nScore = -fLoss(vY, vYhat, nAggre, nConserv) #The highr the better. So negative
            elif sScore == 'adj_r2':
                nScore = r2_score(vY, vYhat)
                #nScore = model.score(mXpart, vY)
                nScore = 1.0 - ( (1.0 - nScore) * (mXpart.shape[0] - 1.0) / (mXpart.shape[0] - len(lFeatures)- 1.0) ) #Adj R2 = 1 - (1 - R2) * (n-1)/n-k-1
            else:#Exception handeling
                raise ValueError('Invalide model argument.for sScore! Your sScore is: %s' %sScore)

            lCandidateScores.append((nScore, nCandidateFea))

        lCandidateScores.sort(reverse=True)
        nBestScore, nBestCandidateFea = lCandidateScores[0]#Choose the highest score and feature
        if nCurrentScore < nBestScore:
            lCandidateFea.remove(nBestCandidateFea)
            lSelectedFea.append(nBestCandidateFea)
            nCurrentScore = nBestScore



    return model, lSelectedFea, nCurrentScore

## Execute the miassing value imputation and model evaluation

In [ ]:
print('Loading data...')
datafit = sio.loadmat('datafit.mat')
X = datafit['X']
Y = np.squeeze(datafit['Y'])


print('Loaded X matrix shape:', X.shape)
print('Loaded Y vector shape:', Y.shape)

print('Number NaN values for each column after imputation')
print(np.sum(np.isnan(X) , axis = 0) )#Check mssing number of each column

#Removing extreme values
nQl = np.percentile(Y, 0.1)
nQu = np.percentile(Y, 99.9)
index = np.where((Y>=nQl) * (Y<=nQu))
Y = Y[index]
X = X[index]


#Set model arguments
sModelUse = 'EN'
sFileName = 'flsqS1000_200'
nTrainWindowL = 1000 #Influences the running time and preceiceness of each loop
nTestWindowL = 200 #Influences how many loops
nTestStart = nTrainWindowL #This is the first date in the current test window

mResultIn = np.empty([1,4])
mResult = np.empty([1,4])
mResult = np.delete(mResult, 0, 0)
mResultIn = np.delete(mResultIn, 0, 0)
lFeatures = []

#Moving window back testing
while(nTestStart <= X.shape[0]):
    mXtrain = X[nTestStart - nTrainWindowL : nTestStart, :]
    mYtrain = Y[nTestStart - nTrainWindowL : nTestStart]
    print('Imputing missing values by k-means clustering')
    mXtrain = KMI.kmeans_missing(mXtrain, 10)[2]  # Impute missing data by k-means clustering

    nTestEnd = nTestStart + nTestWindowL
    nTestEnd = nTestEnd if nTestEnd <= X.shape[0] else X.shape[0] #If it reaches the end of the data
    mXtest = X[nTestStart : nTestEnd, :]
    mYtest = Y[nTestStart : nTestEnd]
    mXtest = KMI.kmeans_missing(mXtest, 10)[2]  # Impute missing data by k-means clustering




    #Transformation trail. The results are not good
    nGlobalMin = np.amin(mXtrain) - 0.001
#    mXtrain = np.concatenate((mXtrain, 1 / (mXtrain - nGlobalMin)), axis=1)
#    mXtest = np.concatenate((mXtest, 1 / (mXtest - nGlobalMin) ), axis=1)
    mXtrain = np.concatenate( (mXtrain, mXtrain**2, 1/(mXtrain - nGlobalMin) ), axis=1)
    mXtest = np.concatenate((mXtest, mXtest**2, 1 / (mXtest - nGlobalMin) ), axis=1)

#    mXtrain = np.log(mXtrain - nGlobalMin)
#    mXtest = np.log(mXtest - nGlobalMin)


#    mYtrain = 3 * mXtrain[:, 0] * 3 * mXtrain[:, 1] + 6 * mXtrain[:, 4]  #Only for test purpose
#    mXtrain[:,1] = mYtrain Only for test purpose

    #####
    # Model here
    print('Featu1res selection...')
    model, lFeaS,nScore = ModelEva.fForwardSelectFeature(mXtrain, mYtrain, sScore='fLoss', nAggre = 2, nConserv = 1, sModel = sModelUse)
    lFeaS = sorted(lFeaS)
    print('Chose features: ',lFeaS, '\n', 'Score: ',nScore, '\n' )
    mXtrain = mXtrain[:, lFeaS]
    mXtest = mXtest[:, lFeaS]
    print('Xshape',mXtrain.shape, mXtest.shape )
    print('Building and training model...')

    model.fit(mXtrain, mYtrain)
    mY_hat = model.predict(mXtrain)
    mY_pred = model.predict(mXtest)

    print('Evaluate in-sample model...')
    nLoss = ModelEva.fLoss(mYtrain, mY_hat, nAggre=2, nConserv=1)
    r2 = r2_score(mYtrain, mY_hat)
    r_value = np.sign(r2) * np.sqrt(abs(r2))
    correlation = np.corrcoef(mYtrain, mY_hat)[0, 1]
    mResultIn = np.concatenate((mResultIn, np.array([[nLoss, r2, r_value, correlation]])), axis=0)
    print('Evaluating in-sample result', mResultIn[-1, :])

    print('Evaluating model...')
    nLoss = ModelEva.fLoss(mYtest, mY_pred, nAggre=2, nConserv=1)
    r2 = r2_score(mYtest, mY_pred)
    r_value = np.sign(r2) * np.sqrt(abs(r2))
    correlation = np.corrcoef(mYtest, mY_pred)[0, 1]
    mResult = np.concatenate((mResult, np.array([[nLoss, r2, r_value, correlation]])), axis=0)
    print('Evaluating result',mResult[-1, :])

    #####
    #Save results for further analysis
    lFeatures = lFeatures.insert(-1, lFeaS)
    with open(sFileName + sModelUse + 'SelectFeatures.csv', 'w') as myfile:
        wr = csv.writer(myfile)
        wr.writerow(lFeatures)

    np.savetxt(sFileName + sModelUse + 'Insamp.csv', mResultIn, delimiter=',')
    np.savetxt(sFileName + sModelUse + 'Outsamp.csv', mResult, delimiter=',')
    nTestStart = nTestEnd